In [1]:
import numpy as np
import pandas as pd
from TFT import TFT

2024-10-02 15:16:22.260792: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tft = TFT(cache_data=False)

Create simulated data

In [3]:
n_days = 100
n_daily_vars = 5

df_daily = pd.DataFrame(
    np.random.randn(n_days, n_daily_vars),
    index=pd.date_range(start="2024-01-01", periods=n_days, freq="D")
)
df_daily.columns = [f"daily_var{i}" for i in range(n_daily_vars)]
df_daily

,daily_var0,daily_var1,daily_var2,daily_var3,daily_var4
2024-01-01,-0.495368,1.354366,0.042015,0.134995,0.723933
2024-01-02,0.425150,-0.678571,-1.296702,0.506769,1.019666
2024-01-03,-1.277709,-0.224408,-1.379400,0.870327,-0.555613
2024-01-04,-0.368318,-1.030343,1.136388,0.454262,-0.015251
2024-01-05,1.885187,-0.854652,0.428716,-1.358738,-0.036511
...,...,...,...,...,...
2024-04-05,0.460083,-1.585899,-0.212797,1.136708,1.266317
2024-04-06,-1.091844,0.471975,-0.724779,-0.524533,1.009133
2024-04-07,-0.181135,-0.673323,-0.196964,-0.115822,-0.724885
2024-04-08,-0.906565,0.702372,-0.221252,-0.767171,1.081795


In [4]:
n_months = 8
n_monthly_vars = 4

df_monthly = pd.DataFrame(
    np.random.randn(n_months, n_monthly_vars),
    index=pd.date_range(start="2024-01-01", periods=n_months, freq="ME")
)
df_monthly.columns = [f"monthly_var{i}" for i in range(n_monthly_vars)]
df_monthly

,monthly_var0,monthly_var1,monthly_var2,monthly_var3
2024-01-31,-0.539176,-1.812220,0.069715,0.804454
2024-02-29,0.213245,-1.183212,-0.628215,0.583423
2024-03-31,-1.434220,0.378218,0.491407,-0.617749
2024-04-30,1.189291,-0.627300,0.100821,1.974981
2024-05-31,-1.164507,0.282187,0.059191,-0.050107
2024-06-30,-0.384647,1.815453,1.227965,0.878783
2024-07-31,-0.472446,-1.238366,-0.474905,-0.796944
2024-08-31,-0.317536,-0.457679,1.967872,-0.211807


Now we create the dependent variable

In [5]:
df_y = df_monthly.pop("monthly_var0")
df_y = pd.DataFrame(df_y)
df_y

,monthly_var0
2024-01-31,-0.539176
2024-02-29,0.213245
2024-03-31,-1.434220
2024-04-30,1.189291
2024-05-31,-1.164507
2024-06-30,-0.384647
2024-07-31,-0.472446
2024-08-31,-0.317536


In [6]:
tft.fit(
    X={"D": df_daily, "ME": df_monthly},
    y={"ME": df_y}
)

Processing `X` and `y` data.
  working out the frequencies...
  calculating ...
Processing `X` and `y` data: concluded
hist LSTM SHAPES:
D: (None, None, 8)
ME: (None, None, 8)
Shape of historical data: (None, 365, 8). Shape of future data: (None, None, 8)
SHAPE OF QOUT: ((None, None, 1), [(None, None, 1), (None, None, 1), (None, None, 1)])
SHAPE OF OUTPUTS: (None, None, 3)
Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 70s 5s/step - loss: 0.8934 - val_loss: 2.0040
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 201ms/step - loss: 0.7443 - val_loss: 2.0052


In [7]:
tft.n_features_in_

{'D': 5, 'ME': 4}

In [8]:
tft.model.inputs, tft.model.outputs

([<KerasTensor shape=(None, None, 5), dtype=float32, sparse=False, name=HistInput__freq_D>,
  <KerasTensor shape=(None, None, 4), dtype=float32, sparse=False, name=HistInput__freq_ME>,
  <KerasTensor shape=(None, None, 5), dtype=float32, sparse=False, name=FutInput__freq_D>],
 [<KerasTensor shape=(None, None, 3), dtype=float32, sparse=False, name=keras_tensor_37>])

In [9]:
tft.predict(
    X={"D": df_daily, "ME": df_monthly},
    y={"ME": df_y}
)

AttributeError: 'dict' object has no attribute 'iloc'

In [ ]:
# def arrange_Xy(X, y):
#     pred_data = {"X_hist": {}, "X_fut": {}, "y": y}
#     for Xk, Xv in X.items():
#         pred_data["X_hist"][Xk] = []
#         for y_date in y.index:
            
    